In [2]:
import sys
import os

# Get the parent directory of the "notebook" folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to the Python path
sys.path.append(project_root)

In [3]:
from strategies.moving_average import MovingAverage
from strategies.momentum import MomentumStrategy
from utils.backtest import Backtest
from utils.data_handler import DataHandler
import yfinance as yf
import pandas as pd
import matplotlib as plt

In [ ]:
import pandas as pd
from itertools import product

# Define tickers and parameter grid
tickers = tickers = [
    "AAPL", "ABT", "ADI", "ADM", "ADP", "ADSK", "AEE", "AEP", "AJG", "AKAM",
    "ALB", "AMAT", "AMD", "AME", "AMGN", "AMT", "AMZN", "AON", "AOS", "APA",
    "APD", "APH", "ARE", "AVB", "AVY", "AXP", "AZO", "BA", "BAC", "BAX",
    "BBY", "BDX", "BEN", "BIIB", "BK", "BLK", "BMY", "BSX", "BXP", "C",
    "CAG", "CAH", "CAT", "CB", "CCI", "CDNS", "CHD", "CHRW", "CINF", "CL",
    "CLX", "CMI", "CNP", "COO", "COP", "COST", "CPB", "CPRT", "CSCO", "CTAS",
    "CTSH", "D", "DE", "DGX", "DHI", "DIS", "DLTR", "DOV", "DRI", "DTE",
    "DVA", "EA", "EBAY", "ECL", "ED", "EFX", "EIX", "EL", "EMN", "EMR",
    "EOG", "EQR", "ES", "ESS", "F", "FAST", "FCX", "FDX", "FE", "FFIV",
    "FITB", "FRT", "GD", "GE", "GILD", "GIS", "GPC", "GWW", "HAL", "HAS"
]  # Extend list as needed

# Parameter grid
short_windows = [5, 10, 20, 30, 50]
long_windows = [50, 100, 150, 200, 300]
modes = ['crossover', 'single']
invert_flags = [True, False]

# Store results
all_results = {}
best_params = []

# Loop over tickers
for ticker in tickers:
    print(f"\n🔍 Evaluating {ticker}")
    data_handler = DataHandler(ticker=ticker, start_date='2010-01-01', end_date='2024-01-01')
    data = data_handler.fetch_data()

    results = []

    # Test each parameter combination
    for short_w, long_w, mode, invert in product(short_windows, long_windows, modes, invert_flags):
        if mode == 'crossover' and short_w >= long_w:
            continue

        strategy = MovingAverage(short_window=short_w, long_window=long_w, mode=mode, invert_signals=invert)
        backtest = Backtest(data, strategy, plot_results=False)

        try:
            result = backtest.run()
            results.append({
                'Ticker': ticker,
                'Short Window': short_w,
                'Long Window': long_w,
                'Mode': mode,
                'Invert Signals': invert,
                **result  # Includes Final Value, Return, Sharpe Ratio, Max Drawdown, etc.
            })
        except Exception as e:
            print(f"❌ Error for {ticker} | short={short_w} long={long_w} mode={mode} invert={invert}: {e}")
            continue

    # Save all results for this ticker
    all_results[ticker] = results

    # Create DataFrame and find best based on Sharpe Ratio
    if results:
        df = pd.DataFrame(results)
        best_row = df.loc[df['Sharpe Ratio'].idxmax()]
        best_params.append(best_row)

        # Optional: display the per-stock comparison table
        print(df[['Short Window', 'Long Window', 'Mode', 'Invert Signals', 'Return', 'Sharpe Ratio', 'Max Drawdown']].sort_values(by='Sharpe Ratio', ascending=False).head())
    else:
        print(f"⚠️ No results for {ticker}")

# Compile best strategy per stock
best_params_df = pd.DataFrame(best_params)

print("\n=== ✅ Summary of Best Parameters by Ticker ===")
print(best_params_df[['Ticker', 'Short Window', 'Long Window', 'Mode', 'Invert Signals', 'Sharpe Ratio', 'Return', 'Max Drawdown']].round(4))



[*********************100%***********************]  1 of 1 completed


🔍 Evaluating AAPL


    Short Window  Long Window       Mode  Invert Signals     Return  \
33            10          200  crossover           False  11.457279   
91            50          200  crossover           False  12.278870   
73            30          200  crossover           False  12.089586   
53            20          200  crossover           False  11.145490   
87            50          150  crossover           False  10.188304   

    Sharpe Ratio  Max Drawdown  
33      0.903220     -0.351761  
91      0.897505     -0.466304  
73      0.896187     -0.386550  
53      0.883473     -0.360403  
87      0.847070     -0.405602  

🔍 Evaluating ABT


[*********************100%***********************]  1 of 1 completed


    Short Window  Long Window       Mode  Invert Signals    Return  \
82            50          100  crossover            True  1.971461   
17             5          300  crossover           False  2.760062   
95            50          300  crossover           False  3.317473   
37            10          300  crossover           False  2.424366   
90            50          200  crossover            True  1.272926   

    Sharpe Ratio  Max Drawdown  
82      0.688779     -0.222334  
17      0.656081     -0.287886  
95      0.654528     -0.316117  
37      0.606870     -0.333514  
90      0.582677     -0.246373  

🔍 Evaluating ADI


...

[*********************100%***********************]  1 of 1 completed


    Short Window  Long Window       Mode  Invert Signals    Return  \
86            50          150  crossover            True  7.364470   
82            50          100  crossover            True  5.453641   
68            30          150  crossover            True  5.208422   
48            20          150  crossover            True  3.443120   
64            30          100  crossover            True  2.970378   

    Sharpe Ratio  Max Drawdown  
86      0.593781     -0.625542  
82      0.552094     -0.590565  
68      0.528317     -0.608435  
48      0.466391     -0.687623  
64      0.447497     -0.632291  

🔍 Evaluating APD


[*********************100%***********************]  1 of 1 completed


    Short Window  Long Window       Mode  Invert Signals    Return  \
90            50          200  crossover            True  1.590303   
94            50          300  crossover            True  1.336448   
56            20          300  crossover            True  1.481851   
68            30          150  crossover            True  1.656546   
64            30          100  crossover            True  1.690880   

    Sharpe Ratio  Max Drawdown  
90      0.615548     -0.207744  
94      0.591926     -0.179199  
56      0.584633     -0.176648  
68      0.565545     -0.206401  
64      0.543900     -0.199197  

🔍 Evaluating APH


[*********************100%***********************]  1 of 1 completed
